In [1]:
# Import necessary libraries
import torch
import sys
import os

# Add the path to your sparse_lo module
sys.path.append('/Users/matthew/Documents/Efficient Gaussian Process on Graphs/Efficient_Gaussian_Process_On_Graphs/efficient_graph_gp_sparse/utils_sparse')

# Import your SparseLinearOperator
from sparse_lo import SparseLinearOperator

In [2]:
# Create a simple test sparse matrix
print("Creating a simple sparse CSR tensor...")

# Create a 4x4 sparse matrix with some non-zero elements
# Matrix structure:
# [1, 0, 2, 0]
# [0, 3, 0, 0] 
# [0, 0, 0, 4]
# [5, 0, 0, 6]

indices = torch.tensor([[0, 0, 1, 2, 3, 3], [0, 2, 1, 3, 0, 3]])  # row, col indices
values = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
size = (4, 4)

sparse_coo = torch.sparse_coo_tensor(indices, values, size)
sparse_csr = sparse_coo.to_sparse_csr()

print(f"Sparse CSR tensor:\n{sparse_csr}")
print(f"\nDense version:\n{sparse_csr.to_dense()}")

Creating a simple sparse CSR tensor...
Sparse CSR tensor:
tensor(crow_indices=tensor([0, 2, 3, 4, 6]),
       col_indices=tensor([0, 2, 1, 3, 0, 3]),
       values=tensor([1., 2., 3., 4., 5., 6.]), size=(4, 4), nnz=6,
       layout=torch.sparse_csr)

Dense version:
tensor([[1., 0., 2., 0.],
        [0., 3., 0., 0.],
        [0., 0., 0., 4.],
        [5., 0., 0., 6.]])


/var/folders/0v/vn4885796ql7_mlpq_dc1n9r0000gn/T/ipykernel_15729/825765867.py:16: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:55.)
  sparse_csr = sparse_coo.to_sparse_csr()


In [11]:
x_1 = [0,1,2,3]

sparse_csr[x_1,:]

RuntimeError: Sparse CSR tensors do not have strides

In [3]:
# Create SparseLinearOperator and test basic properties
print("Creating SparseLinearOperator...")

sparse_lo = SparseLinearOperator(sparse_csr)

print(f"Operator size: {sparse_lo.size()}")
print(f"Operator shape: {sparse_lo.shape}")
print(f"Number of non-zero elements: {sparse_csr._nnz()}")

Creating SparseLinearOperator...
Operator size: torch.Size([4, 4])
Operator shape: torch.Size([4, 4])
Number of non-zero elements: 6


In [16]:
sparse_lo_1 = SparseLinearOperator(sparse_csr)

In [21]:
z = sum([3*sparse_lo_1, sparse_lo])

In [20]:
z[x_1, :] @ random_vector

/Users/matthew/Documents/Efficient Gaussian Process on Graphs/Efficient_Gaussian_Process_On_Graphs/venv/lib/python3.11/site-packages/linear_operator/utils/sparse.py:51: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  if nonzero_indices.storage():
/Users/matthew/Documents/Efficient Gaussian Process on Graphs/Efficient_Gaussian_Process_On_Graphs/venv/lib/python3.11/site-packages/linear_operator/utils/sparse.py:66: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:656.)
  res = cls(index_tensor, value_tensor, interp_size)


tensor([ 28.,  24.,  64., 116.])

In [13]:
y = sparse_lo[x_1, :]

In [23]:
# Test if indexing materializes the SumLinearOperator
print("=== Testing if z[x_1, :] materializes the matrix ===")

# First, define random_vector (which is missing in your code)
random_vector = torch.randn(4)
print(f"Random vector: {random_vector}")

# Create the sum operator
z = sum([3*sparse_lo_1, sparse_lo])
print(f"Original z type: {type(z)}")
print(f"Original z shape: {z.shape}")

# Check if z is still a linear operator
from linear_operator.operators import LinearOperator
print(f"Is z a LinearOperator? {isinstance(z, LinearOperator)}")

# Test indexing and check what happens
x_1 = [0, 1, 2, 3]
print(f"\nTesting indexing with x_1 = {x_1}")

try:
    # Perform the indexing operation
    indexed_z = z[x_1, :]
    
    print(f"After indexing - type: {type(indexed_z)}")
    print(f"After indexing - shape: {indexed_z.shape}")
    print(f"Is indexed_z still a LinearOperator? {isinstance(indexed_z, LinearOperator)}")
    
    # Check if it's a dense tensor (BAD - means materialized)
    if isinstance(indexed_z, torch.Tensor):
        print(f"❌ MATERIALIZED! Became a torch.Tensor")
        print(f"Is sparse tensor? {indexed_z.is_sparse}")
        if not indexed_z.is_sparse:
            print(f"❌ DENSE TENSOR - Memory usage will be high!")
        else:
            print(f"✅ Still sparse tensor")
    else:
        print(f"✅ Still a linear operator: {type(indexed_z)}")
    
    # Test the matrix-vector multiplication
    result = indexed_z @ random_vector
    print(f"Result: {result}")
    
except Exception as e:
    print(f"Error during operation: {e}")
    import traceback
    traceback.print_exc()

# Compare memory usage
print(f"\n=== Memory Usage Comparison ===")

# Check memory usage of original sum operator
if hasattr(z, 'sparse_csr_tensor'):
    original_nnz = z.sparse_csr_tensor._nnz()
    print(f"Original sum operator non-zeros: {original_nnz}")

# Check memory usage after indexing
try:
    indexed_z = z[x_1, :]
    if isinstance(indexed_z, torch.Tensor):
        if indexed_z.is_sparse:
            indexed_nnz = indexed_z._nnz()
            print(f"Indexed sparse tensor non-zeros: {indexed_nnz}")
        else:
            total_elements = indexed_z.numel()
            print(f"❌ Dense tensor total elements: {total_elements}")
            print(f"Memory usage increased by factor: {total_elements / original_nnz:.2f}")
    else:
        print(f"Indexed result is still a linear operator")
except:
    pass

# Test with direct sparse tensor for comparison
print(f"\n=== Comparison with direct sparse tensor indexing ===")
sparse_indexed = sparse_csr[x_1, :]
print(f"Direct sparse indexing type: {type(sparse_indexed)}")
print(f"Direct sparse indexing is_sparse: {sparse_indexed.is_sparse}")

=== Testing if z[x_1, :] materializes the matrix ===
Random vector: tensor([-2.2472, -1.2159, -0.3257,  0.5080])
Original z type: <class 'linear_operator.operators.sum_linear_operator.SumLinearOperator'>
Original z shape: torch.Size([4, 4])
Is z a LinearOperator? True

Testing indexing with x_1 = [0, 1, 2, 3]
After indexing - type: <class 'linear_operator.operators.sum_linear_operator.SumLinearOperator'>
After indexing - shape: torch.Size([4, 4])
Is indexed_z still a LinearOperator? True
✅ Still a linear operator: <class 'linear_operator.operators.sum_linear_operator.SumLinearOperator'>
Result: tensor([-11.5942, -14.5903,   8.1276, -32.7518])

=== Memory Usage Comparison ===
Indexed result is still a linear operator

=== Comparison with direct sparse tensor indexing ===


RuntimeError: Sparse CSR tensors do not have strides

In [25]:
# Test matrix-vector multiplication directly
print("\n=== Testing Matrix-Vector Operations ===")

# Test if @ operator materializes
z = sum([3*sparse_lo_1, sparse_lo])
random_vector = torch.randn(4)

print(f"Before @ operation - z type: {type(z)}")

# This is what happens in your kernel during training
result = z @ random_vector
print(f"After z @ vector - result type: {type(result)}")
print(f"Result: {result}")

# Test matrix-matrix multiplication (this might be the problem)
print("\n=== Testing Matrix-Matrix Operations ===")
random_matrix = torch.randn(4, 3)

try:
    result_mm = z @ random_matrix
    print(f"After z @ matrix - result type: {type(result_mm)}")
    print(f"Result shape: {result_mm.shape}")
except Exception as e:
    print(f"Matrix-matrix multiplication failed: {e}")

# Test transpose operations
print("\n=== Testing Transpose Operations ===")
try:
    z_t = z.t()
    print(f"After transpose - type: {type(z_t)}")
    result_t = z_t @ random_vector
    print(f"After transpose @ vector - result type: {type(result_t)}")
except Exception as e:
    print(f"Transpose operation failed: {e}")

# Test what happens with larger matrices (closer to your real scenario)
print("\n=== Testing with Larger Matrices ===")
# Create a larger sparse matrix to simulate your real case
large_indices = torch.tensor([[0, 1, 2, 100, 200], [50, 60, 70, 150, 250]])
large_values = torch.tensor([1.0, 2.0, 3.0, 4.0, 5.0])
large_size = (1000, 1000)  # Much larger

large_sparse = torch.sparse_coo_tensor(large_indices, large_values, large_size).to_sparse_csr()
large_slo = SparseLinearOperator(large_sparse)
large_z = sum([3*large_slo, large_slo])

print(f"Large matrix z type: {type(large_z)}")
print(f"Large matrix z shape: {large_z.shape}")

# Test indexing on large matrix
test_indices = list(range(100))  # First 100 rows
try:
    large_indexed = large_z[test_indices, :]
    print(f"Large matrix indexing - type: {type(large_indexed)}")
    print(f"Large matrix indexing - shape: {large_indexed.shape}")
    
    # Test if this triggers materialization
    if isinstance(large_indexed, torch.Tensor) and not large_indexed.is_sparse:
        print(f"❌ LARGE MATRIX MATERIALIZED!")
        print(f"Total elements: {large_indexed.numel()}")
    else:
        print(f"✅ Large matrix indexing still lazy")
        
except Exception as e:
    print(f"Large matrix indexing failed: {e}")
    import traceback
    traceback.print_exc()


=== Testing Matrix-Vector Operations ===
Before @ operation - z type: <class 'linear_operator.operators.sum_linear_operator.SumLinearOperator'>
After z @ vector - result type: <class 'torch.Tensor'>
Result: tensor([  1.7275,   9.7564,   5.1427, -19.6442])

=== Testing Matrix-Matrix Operations ===
After z @ matrix - result type: <class 'torch.Tensor'>
Result shape: torch.Size([4, 3])

=== Testing Transpose Operations ===
After transpose - type: <class 'linear_operator.operators.sum_linear_operator.SumLinearOperator'>
After transpose @ vector - result type: <class 'torch.Tensor'>

=== Testing with Larger Matrices ===
Large matrix z type: <class 'linear_operator.operators.sum_linear_operator.SumLinearOperator'>
Large matrix z shape: torch.Size([1000, 1000])
Large matrix indexing - type: <class 'linear_operator.operators.sum_linear_operator.SumLinearOperator'>
Large matrix indexing - shape: torch.Size([100, 1000])
✅ Large matrix indexing still lazy


In [21]:
slo1 = SparseLinearOperator(sparse_csr)
slo2 = SparseLinearOperator(sparse_csr)
# Test addition
print("Testing addition of SparseLinearOperators...")
slo_sum = slo1 + slo2
print(f"Sum operator size: {slo_sum.size()}")
print(f"Sum operator shape: {slo_sum.shape}")

# test multiplication
print("Testing multiplication of SparseLinearOperators...")
slo_prod = slo1 @ slo2
print(f"Product operator size: {slo_prod.size()}")
print(f"Product operator shape: {slo_prod.shape}")

# Create vector with gradient tracking enabled
vec = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
print(f"Operating on vector {vec}...")
result = slo_prod @ vec
print(f"Result of operation: {result}") 

# Test autograd
print("Testing autograd with SparseLinearOperator...")
# Clear any existing gradients
if vec.grad is not None:
    vec.grad.zero_()

result = slo_prod @ vec
result.sum().backward()
print(f"Gradient of the vector: {vec.grad}")

# Verify the autograd functionality by calculating the gradient manually
# For (A @ B) @ vec, gradient w.r.t vec is (A @ B)^T @ ones_vector
# Since we're taking sum().backward(), the upstream gradient is a vector of ones
with torch.no_grad():
    # Get the dense matrices for manual calculation
    A_dense = slo1.sparse_csr_tensor.to_dense()
    B_dense = slo2.sparse_csr_tensor.to_dense()
    AB_dense = A_dense @ B_dense
    ones_vector = torch.ones_like(result)
    expected_grad = AB_dense.t() @ ones_vector

print(f"Expected gradient: {expected_grad}")
print(f"Gradient matches expected: {torch.allclose(vec.grad, expected_grad, atol=1e-6)}")

Testing addition of SparseLinearOperators...
Sum operator size: torch.Size([4, 4])
Sum operator shape: torch.Size([4, 4])
Testing multiplication of SparseLinearOperators...
Product operator size: torch.Size([4, 4])
Product operator shape: torch.Size([4, 4])
Operating on vector tensor([1., 2., 3., 4.], requires_grad=True)...
Result of operation: tensor([ 39.,  18., 116., 209.], grad_fn=<MatmulBackward>)
Testing autograd with SparseLinearOperator...
Gradient of the vector: tensor([56.,  9., 12., 68.])
Expected gradient: tensor([56.,  9., 12., 68.])
Gradient matches expected: True
